Entropy is the qualtification of level of disorder. 

a High Entropy means high disorder.

In information theory, the less entropy you have the more information you have.

At initialization, a ANN is at high state of entropy. When training, the ANN weights are at low entropy whichmeans they managed to learn information from data.

the formula and link between entropy and information is :

Information = - log2(P) // with P the probability of an event occuring
// if P=1 the evnent is certain and we will  no gain any Information from it so the Information = 0
// P = 0 is impossible and the Information resulting is infinite, which mean it will requestion our understanding and world modelm